# ДЗ №3 
## Обучение моделей глубокого обучения на PyTorch

In [1]:
# !pip3 install torch torchvision numpy matplotlib

In [2]:
# !pip install --upgrade matplotlib

  Using cached matplotlib-3.3.3-cp37-cp37m-win_amd64.whl (8.5 MB)


In [1]:
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt

from typing import Tuple, List, Type, Dict, Any

import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import FashionMNIST
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
%matplotlib inline

Ваша задача на этой неделе - повторить модель трёхслойного перцептрона из прошолго задания на **PyTorch**, разобрать лучшие практики обучения моделей глубокого обучения и провести серию экспериментов

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

Для того, чтобы эксперимент можно было повторить, хорошей практикой будет зафиксировать генератор случайных чисел. Также, рекоммендуется зафиксировать RNG в numpy и, если в качестве бэкенда используется cudnn - включить детерминированный режим.

Подробнее: https://pytorch.org/docs/stable/notes/randomness.html

In [3]:
torch.manual_seed(0)
np.random.seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

### Модель

Основным способом организации кода на **Pytorch** является модуль. Простые модели могут быть реализованны из готовых модулей ( к примеру, `torch.nn.Sequential`, `torch.nn.Linear` и т.д. ), для более сложных архитектур часто приходтся реализовывать собственные блоки. Это достаточно легко сделать - достаточно написать класс, наследуемый от `torch.nn.Module` и реализующий метод `.forward`, который принимает и возвращает тензоры ( `torch.Tensor` )

Пример реализации кастомного модуля из официальной документации: https://pytorch.org/tutorials/beginner/pytorch_with_examples.html3

#### Задание 1

Повторите реализацию трёхслойного перцептрона из предыдущего задания на **Pytorch**. Желательно также, чтобы реализация модели имела параметризуемую глубину ( количество слоёв ), количество параметров на каждом слое и функцию активации. Отсутствие такой возможности не снижает балл, но сильно поможет в освоении принципов построения нейросетей с применением библиотеки pytorch.

In [4]:
class Perceptron(torch.nn.Module):
    
    def __init__(self, 
                 input_resolution: Tuple[int, int] = (28, 28),
                 input_channels: int = 1, 
                 hidden_layer_features: List[int] = [256, 256],
                 activation: Type[torch.nn.Module] = torch.nn.ReLU,
                 num_classes: int = 10):

        super( ).__init__()
        

        self.input_layer = nn.Linear(input_resolution[0]*input_resolution[1], 256)
        self.hidden_layers  = nn.ModuleList()
        self.drop_layer = nn.Dropout(p=0.2)
        
        for i in range(len(hidden_layer_features)-1):
            self.hidden_layers.append(nn.Linear(hidden_layer_features[i], hidden_layer_features[i+1]))
        
        self.output_layer = nn.Linear(256, num_classes)
        
        self.sigmoid = nn.Sigmoid()
        
    
    def forward(self, x):

        input_image = x.view(x.size(0), -1)
        
        output = self.input_layer(input_image)
        output = F.relu(output)
        output = self.drop_layer(output)
        
        for layer in self.hidden_layers:
            output = layer(output)
            output = F.relu(output)
            output = self.drop_layer(output)
        
        # Get predictions
        output = self.output_layer(output)
        return output
    
        raise NotImplementedError

Следующий код позволяет посмотреть архитектуру получившейся модели и общее количество обучаемых параметров. Мы хотим, чтобы количество параметров в модели было порядка сотен тысяч. Если у вас получается больше или меньше, попробуйте изменить архитектуру модели.

In [5]:
model = Perceptron()
print(model)
print('Total number of trainable parameters', 
      sum(p.numel() for p in model.parameters() if p.requires_grad))

Perceptron(
  (input_layer): Linear(in_features=784, out_features=256, bias=True)
  (hidden_layers): ModuleList(
    (0): Linear(in_features=256, out_features=256, bias=True)
  )
  (drop_layer): Dropout(p=0.2, inplace=False)
  (output_layer): Linear(in_features=256, out_features=10, bias=True)
  (sigmoid): Sigmoid()
)
Total number of trainable parameters 269322


### Обучающая выборка

На практике, наиболее важным для успеха обучения любой модели машинного обучения является этап подготовки данных. Модели глубокого обучения не являются исключением. Большая, чистая, репрезентативная и релевантная поставленной задаче обучающая выборка часто важнее, чем архитектура самой модели. В предлагаемой задаче используется качественный и проверенный временем MNIST. Однако в практических задачах часто будет получаться так, что лучшим способом добиться улучшения качества модели будет сбор дополнительных данных и очистка исходных данных.

### Предобработка данных

Для улучшения сходимости алгоритма обучения и качества полученной модели данные могут быть предварительно обработаны:

1. Среднее каждой входной переменной близко к нулю
2. Переменные отмасштабированы таким образом, что их дисперсии примерно одинаковы ( из соображений вычислительной устойчивости, мы хотим, чтобы все величины по порядку величины были близки к еденице )
3. По возможности, входные переменные не должны быть скоррелированны. Важнось этого пункта в последние годы ставится под сомнение, но всё-же в некоторых случаях это может влиять на результат

Подробнее можно почитать здесь: http://yann.lecun.com/exdb/publis/pdf/lecun-98b.pdf

### Аугментация (искусственное дополнение) обучающей выборки

В зависимости от задачи можно применять к признаковому описанию объектов обучающей выборки различные преобразования, которые позволят увеличить эффективный размер выборки без дополнительной разметки. К примеру, для задачи классификации кошек и собак можно зеркально отразить изображение вокруг вертикальной оси - при этом класс изображения не изменится, а само изображение останется по прежнему будет принадлежать исходному распределению. Есть много разных техник аугментации, и их применимость и эффективность сильно зависит от данных и задачи.

Подробнее можно почитать здесь: https://link.springer.com/content/pdf/10.1186/s40537-019-0197-0.pdf

### Задание 2

Обоснуйте, почему аугментация обучающей выборки позволяет добиться прироста качества модели, несмотря на то, что она не добавляет в неё дополнительную информацию.

#### Потому что таким образом, мы пытаемся снизить переобученность так как наша нейронная модель будет "пытаться" обобщить свои выводы и "стараться" снизить влияние искажающих или ненужных факторов.  Например: может перестают обращать внимание на поворот картинки, фон, цвет и прочее.  Или в другой формулировке - мы не дает новой информации, но говорим, на что в этом случаее надо обращать внимание.


#### Так же, аугментация может использоваться для акцента на тех количественно небольших случаев, с которыми модель плохо справилась, чтобы "доучиться". То есть мы "раздуваем" количество сложных/редких случаев. 

### Задание 3

Какие осмысленные аугментации вы можете придумать для следующих наборов данных:

1. Набор изображений животных, размеченый на виды животных
2. Набор аудиозаписей голоса, размечеными на языки говорящего
3. Набор cо показаниями датчиков температуры, влажности и давления с одной из метеостанций, размеченый на признак наличия осадков

#Your text here
1. Отражение, вращение, кручение, сдвиг - Принцип такой: зебра, смотрящая налево, остается зеброй, если её отразить по вертикали.
2. Добавление фонового звука. 
3. Добавить шум. 


### Задание 4

Напишите пайплайн для предобработки и аугументации данных. В `torchvision.transforms` есть готовые реализации большинства распространённых техник. Если вы хотите добавить что-то своё, вы можете воспользоваться `torchvision.transforms.Lambda`. При этом следует понимать, что если нужно оценить качество модели на оригинальных данных, пайплайн предварительной обработки данных валидационной выборки не должен включать аугментаций. Следует помнить, однако, что существует подход аугментации данных в момент применения модели (test-time augmentation), который позволяет повысить качество модели в режиме исполнения.

Одним из обязательных шагов в вашем пайплайне должна быть конвертация данных в тензоры Pytorch (`torch.Tensor`): `torchvision.transforms.ToTensor()`.

In [6]:
train_transforms = torchvision.transforms.Compose([  # your core here
    torchvision.transforms.RandomHorizontalFlip(p=0.5),
    torchvision.transforms.RandomVerticalFlip(p=0.5),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

val_transforms = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [7]:
train_dataset = torchvision.datasets.MNIST(root='./mnist', 
                                           train=True, 
                                           download=True,
                                           transform=train_transforms)

val_dataset = torchvision.datasets.MNIST(root='./mnist', 
                                         train=False, 
                                         download=True, 
                                         transform=val_transforms)

Extracting ./mnist\MNIST\raw\train-images-idx3-ubyte.gz to ./mnist\MNIST\raw


Extracting ./mnist\MNIST\raw\train-labels-idx1-ubyte.gz to ./mnist\MNIST\raw


Extracting ./mnist\MNIST\raw\t10k-images-idx3-ubyte.gz to ./mnist\MNIST\raw



Extracting ./mnist\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./mnist\MNIST\raw
Processing...


C:\Users\sasha\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:457: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Done!


In [8]:
val_dataset

Dataset MNIST
    Number of datapoints: 10000
    Root location: ./mnist
    Split: Test
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

Перед тем как запускать обучение всегда стоит посмотреть на данные после предобработки, и удостовериться, что они соответствуют ожидаемым

In [ ]:
indices = np.random.randint(0, len(train_dataset), size=256)

fig, axes = plt.subplots(nrows=16, ncols=16, figsize=(32, 32))
for i, row in enumerate(axes):
    for j, ax in enumerate(row):
        sample_index = indices[i*16+j]
        sample, label = train_dataset[sample_index]
        ax.imshow(sample.cpu().numpy().transpose(1, 2, 0))
        ax.set_title(label.item())

### Обучение модели

Теперь, когда мы реализовали модель и подготовили данные мы можем приступить к непосредственному обучению модели. Костяк функции обучения написан ниже, далее вы должны будете реализовать ключевые части этого алгоритма

In [10]:
from tqdm.notebook import trange, tqdm

def train_model(model: torch.nn.Module, 
                train_dataset: torch.utils.data.Dataset,
                val_dataset: torch.utils.data.Dataset,
                loss_function: torch.nn.Module = torch.nn.CrossEntropyLoss(),
                optimizer_class: Type[torch.optim.Optimizer] = torch.optim,
                optimizer_params: Dict = {},
                initial_lr = 0.01,
                lr_scheduler_class: Any = torch.optim.lr_scheduler.ReduceLROnPlateau,
                lr_scheduler_params: Dict = {},
                batch_size = 64,
                max_epochs = 50,
                early_stopping_patience = 20):
    
    
    optimizer = torch.optim.Adam(model.parameters(), lr=initial_lr, **optimizer_params)
    lr_scheduler = lr_scheduler_class(optimizer, **lr_scheduler_params)
    
    train_loader = torch.utils.data.DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size)

    best_val_loss = None
    best_epoch = None
    epoch_train_losses = []
    
    #pbar = tqdm(total=max_epochs)
    for epoch in range(max_epochs) :
        
        print(f'Epoch {epoch}')
        loss_train = train_single_epoch(model, optimizer, loss_function, train_loader)
        val_metrics = validate_single_epoch(model, loss_function, val_loader)
        print(f'Validation metrics: \n{val_metrics}')

        lr_scheduler.step(val_metrics['loss'])
        
        if best_val_loss is None or best_val_loss > val_metrics['loss']:
            print(f'Best model yet, saving')
            best_val_loss = val_metrics['loss']
            best_epoch = epoch
            torch.save(model, './best_model.pth')
            
        if epoch - best_epoch > early_stopping_patience:
            print('Early stopping triggered')
            return
        
        #pbar.update(1)
        #pbar.set_postfix({'loss': val_metrics})
    
    #tqdm.close()
        #epoch_train_losses.append(loss_train)
        #epoch_train_losses.append(val_metrics['loss'])
        #plt.plot(epoch_train_losses)
        #plt.show()

### Задание 5

Реализуйте функцию, производящую обучение сети на протяжении одной эпохи ( полного прохода по всей обучающей выборке ). На вход будет приходить модель, оптимизатор, функция потерь и объект типа `DataLoader`. При итерировании по `data_loader` вы будете получать пары вида ( данные, целевая_переменная )

In [11]:
def train_single_epoch(model: torch.nn.Module,
                       optimizer: torch.optim.Optimizer, 
                       loss_function: torch.nn.Module, 
                       data_loader: torch.utils.data.DataLoader,
                       max_epochs = 1):
    
    epoch_train_losses = []
    #print(len(data_loader))
    for X_batch, y_batch in data_loader:
        
        y_pred = model(X_batch)
        loss = loss_function(y_pred, y_batch)

        # зануляем градиент
        optimizer.zero_grad()

        # backward
        loss.backward()

        # ОБНОВЛЯЕМ веса
        optimizer.step()
        epoch_train_losses.append(loss.item())    
        # Запишем число (не тензор) в наши батчевые лоссы
    return np.mean(epoch_train_losses)
 
#     plt.plot(epoch_train_losses)
#     plt.show()

### Задание 6

Реализуйте функцию производящую вычисление функции потерь на валидационной выборке.  На вход будет приходить модель, функция потерь и `DataLoader`. На выходе ожидается словарь с вида:
```
{
    'loss': <среднее значение функции потерь>,
    'accuracy': <среднее значение точности модели>
}
```

In [12]:
from sklearn.metrics import accuracy_score

def validate_single_epoch(model: torch.nn.Module,
                          loss_function: torch.nn.Module, 
                          data_loader: torch.utils.data.DataLoader):
    
    D = {}
    losses = []
    accuracies = []
    for i, data in enumerate(data_loader, 0):
        inputs, labels = data
        #print(inputs.shape, labels.shape)
        #inputs, labels = data[0].to(device), data[1].to(device)  
        
        outputs = model(inputs)
        
        loss = loss_function(outputs, labels)
        
        losses.append(loss.item()) #loss_function(x,y)
        
        labels_pred = np.argmax(outputs.detach().numpy(), axis=1)
        
        accuracy = accuracy_score(labels_pred, labels)
        accuracies.append(accuracy)
    #print(np.mean(losses))
    
    D['loss'] = np.mean(losses)
    D['accuracy'] = np.mean(accuracies)
    return D

Если вы корректно реализовали все предыдущие шаги и ваша модель имеет достаточное количество обучаемых параметров, то в следующей ячейке должен пойти процесс обучения, и мы должны достичь итоговой точности (в смысле меры accuracy, доли верных ответов) выше 90%

In [13]:
#model.to(device)
train_model(model, 
            train_dataset=train_dataset, 
            val_dataset=val_dataset, 
            loss_function=torch.nn.CrossEntropyLoss(), 
            initial_lr=0.0004)

Epoch 0
Validation metrics: 
{'loss': 0.5523548189811646, 'accuracy': 0.8190684713375797}
Best model yet, saving
Epoch 1


D:\Users\sasha\Anaconda3\lib\site-packages\torch\serialization.py:241: UserWarning: Couldn't retrieve source code for container of type Perceptron. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Validation metrics: 
{'loss': 0.4103707963968538, 'accuracy': 0.8683320063694268}
Best model yet, saving
Epoch 2
Validation metrics: 
{'loss': 0.39059766246729594, 'accuracy': 0.8729100318471338}
Best model yet, saving
Epoch 3
Validation metrics: 
{'loss': 0.34966683577580054, 'accuracy': 0.8912221337579618}
Best model yet, saving
Epoch 4
Validation metrics: 
{'loss': 0.3430380040103463, 'accuracy': 0.88953025477707}
Best model yet, saving
Epoch 5
Validation metrics: 
{'loss': 0.2928224984959812, 'accuracy': 0.9075437898089171}
Best model yet, saving
Epoch 6
Validation metrics: 
{'loss': 0.2874333345016856, 'accuracy': 0.9077428343949044}
Best model yet, saving
Epoch 7
Validation metrics: 
{'loss': 0.2580317619262607, 'accuracy': 0.918093152866242}
Best model yet, saving
Epoch 8
Validation metrics: 
{'loss': 0.28414101153612137, 'accuracy': 0.9093351910828026}
Epoch 9
Validation metrics: 
{'loss': 0.24769758668010403, 'accuracy': 0.9214769108280255}
Best model yet, saving
Epoch 10
Vali

### Задание 7

Модифицируйте процесс обучения таким образом, чтобы достигнуть наилучшего качества на валидационной выборке. Модель должна оставаться N-слойным перцептроном с количеством обучаемых параметров <= 500000. Для обучения разрешается использовать только набор данных MNIST. Процесс обучения вы можете изменять по собственному усмотрению. К примеру, вы можете менять:

* Архитектуру модели в рамках наложенных ограничений на количество параметров и вид архитектуры (многослойный перцептрон)
* Функции активации в модели
* Используемый оптимизатор
* Расписание шага оптимизации
* Сэмплинг данных при обучении ( e.g. hard negative mining)

В результате мы ожидаем увидеть код экспериментов и любые инсайты, которые вы сможете получить в процессе

# Обучил улучшенную модель до 90% и выше

(Последняя эпоха дала 95.2%)

Заметил, что классы 6 и 9 можно отражать по вертикале, чтобы сгенерить дополнительные новые данные для этих классов, а класс 8 можно отражать по вертикали и горизонтале при аугументации 